# E-Commerce Order Fulfillment – Shipping Timeline & Data Cleaning

This notebook documents how we clean and adjust the **E-Commerce Order Fulfillment Dataset (50K Records)** for use in the project:

> **End-to-End Delivery & E-Commerce Logistics Analysis using Python and Power BI**

We focus on one dataset only (no external logistics table) and:

1. Inspect raw order, ship, and delivery dates.
2. Show that the original data has unrealistic gaps
3. Apply a realistic **domestic e-commerce shipping rule**:

Let:

- `total_cycle_days = delivery_date - order_date`

Then define dispatch delay (order → ship):

- If `total_cycle_days <= 4` → **same-day shipping** (`0` days delay)  
- If `5 <= total_cycle_days <= 8` → **next-day shipping** (`1` day delay)  
- If `total_cycle_days > 8` → **3 days delay**  

We use this to recompute:

- `ship_date`
- `delivery_days = delivery_date - ship_date`

The final cleaned dataset is written by `src/analysis.py` and used for Power BI.
This notebook is here to **explain and justify** the transformation.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

BASE_DIR = Path("..").resolve()
DATA_RAW_DIR = BASE_DIR / "data" / "raw"

orders_path = DATA_RAW_DIR / "E-Commerce Order Fulfillment Dataset (50K Records).csv"
orders_path


In [ ]:
df_raw = pd.read_csv(orders_path)
df_raw.head()


In [ ]:
print("Shape:", df_raw.shape)
print("\nColumns:\n", df_raw.columns.tolist())

df_raw.info()


In [ ]:
df = df_raw.copy()
df.columns = [c.strip() for c in df.columns]

rename_map = {
    "Order_ID": "order_id",
    "Customer_Region": "customer_region",
    "Product_Category": "product_category",
    "Order_Date": "order_date",
    "Ship_Date": "ship_date",
    "Delivery_Date": "delivery_date",
    "Shipping_Mode": "shipping_mode",
    "Shipping_Cost": "shipping_cost",
    "Delivery_Status": "delivery_status",
    "Delivery_Days": "delivery_days",
}
df = df.rename(columns=rename_map)

for col in ["order_date", "ship_date", "delivery_date"]:
    df[col] = pd.to_datetime(df[col], errors="coerce")

df[["order_date", "ship_date", "delivery_date"]].head()


In [ ]:
# Raw time differences before applying our rule
mask = df["order_date"].notna() & df["ship_date"].notna() & df["delivery_date"].notna()

raw_dispatch_days = (df.loc[mask, "ship_date"] - df.loc[mask, "order_date"]).dt.days
raw_delivery_days = (df.loc[mask, "delivery_date"] - df.loc[mask, "ship_date"]).dt.days
raw_total_cycle = (df.loc[mask, "delivery_date"] - df.loc[mask, "order_date"]).dt.days

print("Raw dispatch (order -> ship) days:")
print(raw_dispatch_days.describe())

print("\nRaw delivery (ship -> delivery) days:")
print(raw_delivery_days.describe())

print("\nRaw total cycle (order -> delivery) days:")
print(raw_total_cycle.describe())


## Adjusting Shipping Dates – Domestic E-Commerce Rule

To make the dataset more realistic for a domestic e-commerce operation,
we adjust `ship_date` using the following rule:

Let:

```text
total_cycle_days = delivery_date - order_date


In [ ]:
ship_date = order_date + dispatch_delay
delivery_days = delivery_date - ship_date


In [ ]:

---

## 📌 Cell 8 – Python: define the function (same logic as `analysis.py`)

```python
def adjust_shipping_dates(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    if not {"order_date", "delivery_date"}.issubset(df.columns):
        return df

    mask = df["order_date"].notna() & df["delivery_date"].notna()
    total_cycle = (df.loc[mask, "delivery_date"] - df.loc[mask, "order_date"]).dt.days

    # Replace negative or weird values with median of valid values
    median_cycle = total_cycle[total_cycle >= 0].median()
    total_cycle = total_cycle.where(total_cycle >= 0, median_cycle)

    dispatch_delay_days = np.select(
        [
            total_cycle <= 4,
            (total_cycle > 4) & (total_cycle <= 8),
            total_cycle > 8,
        ],
        [0, 1, 3],
        default=1,
    )

    dispatch_delay = pd.to_timedelta(dispatch_delay_days, unit="D")

    df.loc[mask, "ship_date"] = df.loc[mask, "order_date"] + dispatch_delay
    df.loc[mask, "delivery_days"] = (
        df.loc[mask, "delivery_date"] - df.loc[mask, "ship_date"]
    ).dt.days

    return df


In [ ]:
df_adjusted = adjust_shipping_dates(df)

df_adjusted[["order_date", "ship_date", "delivery_date", "delivery_days"]].head(10)


In [ ]:
mask_adj = df_adjusted["order_date"].notna() & df_adjusted["ship_date"].notna() & df_adjusted["delivery_date"].notna()

new_dispatch_days = (df_adjusted.loc[mask_adj, "ship_date"] - df_adjusted.loc[mask_adj, "order_date"]).dt.days
new_delivery_days = (df_adjusted.loc[mask_adj, "delivery_date"] - df_adjusted.loc[mask_adj, "ship_date"]).dt.days

print("AFTER adjustment – dispatch (order -> ship) days:")
print(new_dispatch_days.describe())

print("\nAFTER adjustment – delivery (ship -> delivery) days:")
print(new_delivery_days.describe())


In [ ]:
df_adjusted["on_time_flag"] = np.where(df_adjusted["delivery_status"].str.lower() == "delayed", 0, 1)
df_adjusted["delay_flag"] = 1 - df_adjusted["on_time_flag"]

print("Overall on-time %:", df_adjusted["on_time_flag"].mean() * 100)

df_adjusted["shipping_mode"].value_counts()


## Summary

In this notebook we:

1. Loaded the **E-Commerce Order Fulfillment Dataset (50K Records)**.
2. Standardised column names and parsed dates.
3. Observed that the raw time gaps between `order_date`, `ship_date`, and `delivery_date`
   were often unrealistic for domestic e-commerce.
4. Applied a clear business rule to adjust `ship_date` (0–3 days after `order_date`)
   based on the total cycle time (order → delivery).
5. Recomputed `delivery_days = delivery_date - ship_date` to keep the timeline consistent.

The production-ready version of this logic lives in `src/analysis.py`, which:

- Cleans the dataset end-to-end
- Exports `data/processed/cleaned_merged_data.csv`
- Generates visuals for Power BI and reporting
